In [1]:
import torch
import matplotlib.pyplot as plt
import json

from lume_model.utils import variables_from_yaml
from lume_model.torch import LUMEModule, PyTorchModel

In [2]:
# load the information required to convert PV to sim and vice versa
with open('info/pv_mapping.json', 'r') as f:
    pv_mapping = json.load(f)
    f.close()

with open('info/model.json', 'r') as f:
    model_info = json.load(f)
    f.close()

In [3]:
class PVtoSimTransformer(torch.nn.Module):
    """Reversible transformer for converting EPICS PV units to simulation units"""
    def __init__(self, conversion: torch.Tensor):
        super().__init__()
        self._conversion = conversion

    def forward(self, x: torch.Tensor):
        return self.transform(x)

    def transform(self, x: torch.Tensor):
        """convert PV units to simulation units"""
        self._conversion = self._conversion.to(x)
        return x * self._conversion

    def untransform(self, x: torch.Tensor):
        """convert simulation units to PV units"""
        self._conversion = self._conversion.to(x)
        return x / self._conversion

In [4]:
# load the conversions from file in the right order
input_conversions = torch.zeros(len(model_info['model_in_list']))
for i, sim_name in enumerate(model_info['model_in_list']):
    pv_name = pv_mapping['sim_name_to_pv_name'][sim_name]
    input_conversions[i] = pv_mapping['pv_to_sim_factor'][pv_name] 

output_conversions = torch.zeros(len(model_info['model_out_list']))
for i, sim_name in enumerate(model_info['model_out_list']):
    pv_name = pv_mapping['sim_name_to_pv_name'][sim_name]
    output_conversions[i] = pv_mapping['pv_to_sim_factor'][pv_name] 

In [5]:
# construct the pv to sim transformers using the conversions
input_pv_to_sim = PVtoSimTransformer(input_conversions)
output_pv_to_sim = PVtoSimTransformer(output_conversions)


In [7]:
# load sim to neural network transformers
input_sim_to_nn = torch.load("model/input_transformer.pt")
output_sim_to_nn = torch.load("model/output_transformer.pt")

In [8]:
# load in- and output variable specification
input_variables, output_variables = variables_from_yaml(open("model/pv_variables.yml"))

In [9]:
# create lume model
lume_model = PyTorchModel(
    model_file="model/model.pt",
    input_variables=input_variables,
    output_variables=output_variables,
    input_transformers=[input_pv_to_sim, input_sim_to_nn],  # add the input_pv_to_sim transformers before the sim to nn transformation
    output_transformers=[output_sim_to_nn, output_pv_to_sim],  # add the output_pv_to_sim transformers after the sim to nn transformation
)

In [10]:
# wrap in lume module
lume_module = LUMEModule(
    model=lume_model,
    feature_order=lume_model.features,
    output_order=lume_model.outputs,
)
lume_module

LUMEModule(
  (base_model): Sequential(
    (0): Linear(in_features=16, out_features=100, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=100, out_features=200, bias=True)
    (3): ELU(alpha=1.0)
    (4): Dropout(p=0.05, inplace=False)
    (5): Linear(in_features=200, out_features=200, bias=True)
    (6): ELU(alpha=1.0)
    (7): Dropout(p=0.05, inplace=False)
    (8): Linear(in_features=200, out_features=300, bias=True)
    (9): ELU(alpha=1.0)
    (10): Dropout(p=0.05, inplace=False)
    (11): Linear(in_features=300, out_features=300, bias=True)
    (12): ELU(alpha=1.0)
    (13): Dropout(p=0.05, inplace=False)
    (14): Linear(in_features=300, out_features=200, bias=True)
    (15): ELU(alpha=1.0)
    (16): Dropout(p=0.05, inplace=False)
    (17): Linear(in_features=200, out_features=100, bias=True)
    (18): ELU(alpha=1.0)
    (19): Dropout(p=0.05, inplace=False)
    (20): Linear(in_features=100, out_features=100, bias=True)
    (21): ELU(alpha=1.0)
    (22): Linear(in_f

In [11]:
# load example data and calculate predictions
inputs_small = input_pv_to_sim.untransform(torch.load("info/inputs_small.pt"))
outputs_small = output_pv_to_sim.untransform(torch.load("info/outputs_small.pt"))
with torch.no_grad():
    predictions = lume_module(inputs_small)

In [12]:
# plot example data and predictions
nrows, ncols = 3, 2
fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(10, 15))
for i, output_name in enumerate(lume_model.outputs):
    ax_i = ax[i // ncols, i % ncols]
    if i < outputs_small.shape[1]:
        sort_idx = torch.argsort(outputs_small[:, i])
        x_axis = torch.arange(outputs_small.shape[0])
        ax_i.plot(x_axis, outputs_small[sort_idx, i], "C0x", label="outputs")
        ax_i.plot(x_axis, predictions[sort_idx, i], "C1x", label="predictions")
        ax_i.legend()
        ax_i.set_title(output_name)
ax[-1][-1].axis('off')
fig.tight_layout()

# Missing Output PVs
If we have missing outputs, for example outputs of the model that are not measured in EPICS, we simply remove these from the **LUMEModule** by not passing them to the `output_order` parameter. **NOTE** They still need to be listed in the PyTorchModel. 

In [13]:
epics_lume_module = LUMEModule(
    model=lume_model,  # we use the same lume_model as before
    feature_order=lume_model.features,
    output_order=lume_model.outputs[0:2],  # here we remove the additional parameters
)
epics_lume_module.output_order

['OTRS:IN20:571:XRMS', 'OTRS:IN20:571:YRMS']

In [14]:
with torch.no_grad():
    predictions = epics_lume_module(inputs_small)
predictions.shape

torch.Size([283, 2])

In [15]:
# plot example data and predictions
nrows, ncols = 1, 2
fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(10, 5))
for i, output_name in enumerate(epics_lume_module.output_order):
    sort_idx = torch.argsort(outputs_small[:, i])
    x_axis = torch.arange(outputs_small.shape[0])
    ax[i].plot(x_axis, outputs_small[sort_idx, i], "C0x", label="outputs")
    ax[i].plot(x_axis, predictions[sort_idx, i], "C1x", label="predictions")
    ax[i].legend()
    ax[i].set_title(output_name)
fig.tight_layout()